In [1]:
!pip install openai

In [23]:
import openai
import json
import re
from typing import List, Dict
from google.colab import userdata

# Set your API key (input this manually)
key = userdata.get('OPENAI_API_KEY')
openai.api_key = key

schema_english = {
    "general information": {
        "name": "string",
        "brand": "string",
        "category": "string",
        "original_price": "float",
        "discount_price": "float",
        "description": "string",
        "design": "string",
        "promotion": "string",
    },
    "technical specifications": {
        "display": ["string"],
        "memory and storage": ["string"],
        "camera": "string",
        "battery": "string",
        "operating system": "string",
        "chip": "string",
        "connectivity": ["string"],
        "audio": "string",
        "charging and expansion": ["string"],
    },
    "extra_metadata": {
        "special feature": ["string"],
        "accessories": ["string"],
    }
}


def extract_and_clean_json_from_response(content: str) -> str:
    """Extract JSON data from a response and remove comments."""
    # Extract JSON content between ```json and ```
    json_match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
    if json_match:
        json_content = json_match.group(1)

        # Remove any comments or lines with `//`
        cleaned_json_content = re.sub(r'//.*', '', json_content)
        return cleaned_json_content
    return None


def convert_to_json(raw_data, schema):
    # Define the prompt
    raw_data = raw_data.strip().rstrip('.').rstrip('...').strip()
    prompt = f"Convert the following raw data into JSON format based on this schema:\n\nSchema: {json.dumps(schema)}\n\nRaw Data: {json.dumps(raw_data)}"

    # Call the OpenAI API
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that formats data."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )

    # Extract and parse the JSON content
    content = response.choices[0].message.content.strip()
    json_content = extract_and_clean_json_from_response(content)

    if json_content:
        try:
            json_data = json.loads(json_content)  # Parse the JSON-formatted text
            return json_data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("No JSON found in response.")
    return None

def process_and_save_data(raw_data_list, output_file):
    schema = schema_english

    processed_data = []

    for raw_data in raw_data_list:
        json_data = convert_to_json(raw_data, schema)
        if json_data:
            processed_data.append(json_data)

    # Save the processed data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

# # Example usage with a list of raw data strings
# raw_data_list = [
#     "Tên sản phẩm: iPhone 13, Thương hiệu: Apple, Giá: 20000000, Danh mục: điện thoại thông minh",
#     "Tên sản phẩm: Galaxy S21, Thương hiệu: Samsung, Giá: 18000000, Danh mục: điện thoại thông minh",
#     "Tên sản phẩm: Xiaomi Mi 11, Thương hiệu: Xiaomi, Giá: 15000000, Danh mục: điện thoại thông minh",
#     "Tên sản phẩm: Oppo Find X3 Pro, Thương hiệu: Oppo, Giá: 22000000, Danh mục: điện thoại thông minh"
# ]

# # Save converted data to 'output_data.json'
# process_and_save_data(raw_data_list, 'output_data.json')


In [ ]:
# import json

# # Open and read the JSON file
# with open('output_data.json', 'r') as file:
#     data = json.load(file)

# # Print the data
# print(data)


[{'general information': {'name': 'iPhone 13', 'brand': 'Apple', 'category': 'điện thoại thông minh', 'price': 20000000.0, 'description': '', 'design': ''}, 'technical specifications': {'display': [], 'memory and storage': [], 'camera': '', 'battery': '', 'operating system': '', 'chip': '', 'connectivity': [], 'audio': '', 'charging and expansion': []}, 'extra_metadata': {'special feature': [], 'accessories': []}}, {'general information': {'name': 'Galaxy S21', 'brand': 'Samsung', 'category': 'điện thoại thông minh', 'price': 18000000.0, 'description': '', 'design': ''}, 'technical specifications': {'display': [], 'memory and storage': [], 'camera': '', 'battery': '', 'operating system': '', 'chip': '', 'connectivity': [], 'audio': '', 'charging and expansion': []}, 'extra_metadata': {'special feature': [], 'accessories': []}}, {'general information': {'name': 'Xiaomi Mi 11', 'brand': 'Xiaomi', 'category': 'điện thoại thông minh', 'price': 15000000.0, 'description': '', 'design': ''}, 

# Raw Data:

In [3]:
import requests
from bs4 import BeautifulSoup

def spider(max_pages):
    page = 1
    while page <= max_pages:
        url = 'https://www.xtmobile.vn/dien-thoai/?p=' + str(page)
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')

        print("Products List:")
        print("================================================")
        # Tìm tất cả các thẻ <div> có class "pinfo" và trong đó có thẻ <h3>
        for pinfo in soup.find_all('div', class_='pinfo'):
            # Lấy thẻ <a> trong thẻ <h3> để lấy link
            link_tag = pinfo.find('h3').find('a')
            price_tag = pinfo.find()
            href = link_tag.get('href')
            title = link_tag.get('title')
            print(f"Product Title: {title}")
            print(f"Product URL: {href}")
            print()

        page += 1

# Chạy hàm để kiểm tra
spider(1)  # Bạn có thể thay đổi số trang nếu muốn lấy nhiều hơn


Products List:
Product Title: Samsung Galaxy S24 5G (8GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s24

Product Title: Samsung Galaxy S24 Plus 5G (12GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s24-plus

Product Title: Samsung Galaxy S24 Ultra 5G (12GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s24-ultra

Product Title: Samsung Galaxy A55 5G (8GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-a55-8gb-256gb-cty

Product Title: Samsung Galaxy S23 Ultra (8GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s23-ultra-8gb-256gb

Product Title: Samsung Galaxy A90 5G (6GB|128GB) SM-A908N (Cũ Đẹp)
Product URL: https://www.xtmobile.vn/samsung-galaxy-a90

Product Title: iPhone 16 256GB (VN/A)
Product URL: https://www.xtmobile.vn/iphone-16-256gb-chinh-hang

Product Title: iPhone 16 Plus 256GB (VN/A)
Product URL: https://www.xtmobile.vn/iphone-16-plus-256gb-chinh-hang

Product Title: iPhone 16 Plu

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def spider(max_pages):
    page = 1
    while page <= max_pages:
        url = 'https://www.xtmobile.vn/dien-thoai/?p=' + str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        print("Products List:")
        print("=================================================")
        # Lặp qua từng sản phẩm
        for pinfo in soup.find_all('div', class_='pinfo'):
            # Lấy link sản phẩm
            link_tag = pinfo.find('h3').find('a')
            href = link_tag.get('href')
            title = link_tag.get('title')

            # Lấy giá sản phẩm
            price_tag = pinfo.find('div', class_='price')
            # print(price_tag)
            # print(price_tag.text)
            # price = price_tag.text.strip() if price_tag else 'N/A'

            # Extract the three prices
            # First price is the main content of div
            current_price = price_tag.contents[0].strip()
            original_price = current_price
            discount_price = '0đ'
            if price_tag.find('span'):
              # Second price is inside the first span
              original_price = price_tag.find('span').contents[0].strip()

              # Third price is inside the nested span with class "mobile-discount"
              discount_price = price_tag.find('span', class_='mobile-discount').text.strip()

            # In thông tin sản phẩm
            print(f"Product Title: {title}")
            print(f"Product URL: {href}")
            # print(f"Price: {price}")
            # Display the prices
            print("Current Price:", current_price)
            print("Original Price:", original_price)
            print("Discount Price:", discount_price)
            print("-" * 30)

        # Thêm độ trễ giữa các request để tránh bị chặn
        time.sleep(2)  # Đợi 2 giây giữa các request
        page += 1

# Chạy hàm để kiểm tra
spider(1)  # Thay đổi số trang tùy ý nếu bạn muốn lấy thêm sản phẩm


Products List:
Product Title: Samsung Galaxy S24 5G (8GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s24
Current Price: 15.599.000đ
Original Price: 22.999.000đ
Discount Price: - 7.400.000đ
------------------------------
Product Title: Samsung Galaxy S24 Plus 5G (12GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s24-plus
Current Price: 17.199.000đ
Original Price: 26.990.000đ
Discount Price: - 9.791.000đ
------------------------------
Product Title: Samsung Galaxy S24 Ultra 5G (12GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-s24-ultra
Current Price: 23.599.000đ
Original Price: 33.999.000đ
Discount Price: - 10.400.000đ
------------------------------
Product Title: Samsung Galaxy A55 5G (8GB|256GB) (CTY)
Product URL: https://www.xtmobile.vn/samsung-galaxy-a55-8gb-256gb-cty
Current Price: 9.399.000đ
Original Price: 9.399.000đ
Discount Price: 0
------------------------------
Product Title: Samsung Galaxy S23 Ultra (8GB|256GB) (CT

In [ ]:
def get_single_meta_item_data(item_url):
  source_code = requests.get(item_url)
  plain_text = source_code.text
  soup = BeautifulSoup(plain_text)

  if len(modal_body_tag := soup.findAll('div', {'class': 'modal-body'})) >= 2:
    meta_data_tag = modal_body_tag[1]
    # print(meta_data_tag.text)
    tmp = meta_data_tag.get_text(separator='|', strip=True)
    print(type(tmp))
  return tmp

# get_single_meta_item_data('https://www.xtmobile.vn/iphone-7-plus-32gb-likenew-99')

In [ ]:
temp = get_single_meta_item_data('https://www.xtmobile.vn/iphone-7-plus-32gb-likenew-99')

<class 'str'>


In [ ]:
print(temp)

Thông tin màn hình|Công nghệ màn hình|LED-backlit IPS LCD|Độ phân giải|Full HD (1080 x 1920 pixels)|Màn hình rộng|5.5"|Mặt kính cảm ứng|Kính oleophobic (ion cường lực)|Kiểu màn hình|Thông tin camera sau|Độ phân giải|2 camera 12 MP|Quay phim|Quay phim 4K 2160p@30fps|Đèn Flash|4 đèn LED (2 tông màu)|Chụp ảnh nâng cao|Tự động lấy nét, Chạm lấy nét, Nhận diện khuôn mặt, HDR, Panorama, Chống rung quang học (OIS)|Thông tin camera trước|Độ phân giải|7 MP|Videocall|Hỗ trợ VideoCall thông qua ứng dụng|Tính năng khác|Retina Flash, Nhận diện khuôn mặt, Quay video Full HD, Tự động lấy nét, Selfie ngược sáng HDR|Hệ điều hành - CPU|Hệ điều hành|iOS|Chipset (hãng SX CPU)|Apple A10 Fusion 4 nhân 64-bit|Tốc độ CPU|2.3 GHz|Chip đồ họa (GPU)|Chip đồ họa 6 nhân|Bộ nhớ & Lưu trữ|RAM|3 GB|Bộ nhớ trong|32 GB|Thẻ sim|Không|Kết nối|Mạng di động|3G, 4G LTE Cat 9|SIM|1 Nano SIM|Wifi|Wi-Fi 802.11 a/b/g/n/ac, Dual-band, Wi-Fi hotspot|GPS|A-GPS, GLONASS|Bluetooth|v4.2, A2DP, LE|Cổng kết nối/sạc|Lightning|Jack 

In [ ]:
def get_single_desc_item_data(item_url):
  source_code = requests.get(item_url)
  plain_text = source_code.text
  soup = BeautifulSoup(plain_text)

  if len(modal_body_tag := soup.findAll('div', {'class': 'content_hide content-desc'})) >= 1:
    product_desc_tag = modal_body_tag[0]
    # print(product_desc_tag.text)

  return product_desc_tag.text

get_single_desc_item_data('https://www.xtmobile.vn/iphone-7-plus-32gb-likenew-99')

iPhone 7 Plus 32GB cũ được ra mắt từ tháng 10 năm 2016 nhưng cho đến bây giờ vẫn nhiều người lựa chọn nó thay vì iPhone Xs hay iPhone Xr, đó là nhờ mức giá siêu rẻ mà ít flagship nào có được.

Thiết kế iPhone 7 Plus 32GB Quốc Tế cũ - Sang, đẹp và chắc chắn
iPhone 7 Plus 32GB cũ bản Quốc Tế mang thiết kế nguyên khối với lớp kính bảo vệ Ion-strengthened Galss ở mặt trước. Máy có khả năng chống nước và bụi bẩn là IP67, tức là bạn có thể ngâm nó ở độ sâu 1 m dưới nước trong 30 phút. Máy khá nhẹ với 188g và kích thước 158.2x77.9x7.3 mm. Phím home ảo vẫn được thiết kế dạng button giống như iPhone 7.

iPhone 7 Plus 32GB Quốc Tế cũ sử dụng màn hình LED-backlit IPS LCD kích thước 5.5 inch với độ phân giải 1080x1920 Pixels, tỉ lệ 16:9 và độ phủ màn hình khoảng 67.7%. Máy hiển thị dưới ánh sáng mặt trời khá tốt nhờ gam màu rộng và độ tương phản đạt 1398:1.

Hiệu năng iPhone 7 Plus 32GB Quốc Tế cũ - Chipset mạnh mẽ, hệ điều hành cập nhật liên tục
Apple đã trang bị cho iPhone 7 Plus 32GB cũ (fullbo

# Save Data to Pandas Dataframe:
```
  --Id
  --Title
  --Original Price
  --Discount
  --Meta_Data
  --Description
```

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_single_meta_item_data(item_url):
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    meta_data_text = "N/A"  # Default in case metadata is not found

    modal_body_tag = soup.find_all('div', class_='modal-body')
    if len(modal_body_tag) >= 2:
        meta_data_tag = modal_body_tag[1]
        meta_data_text = meta_data_tag.get_text(separator=' ', strip=True)

    return meta_data_text

def get_single_desc_item_data(item_url):
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    description_text = "N/A"  # Default in case description is not found

    modal_body_tag = soup.find_all('div', class_='content_hide content-desc')
    if len(modal_body_tag) >= 1:
        product_desc_tag = modal_body_tag[0]
        description_text = product_desc_tag.get_text(separator=' ', strip=True)

    return description_text

def get_single_promotion_data(item_url):
    source_code = requests.get(item_url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    promotion_text = "N/A"  # Default in case promotion data is not found

    # Tìm phần tử chứa khuyến mãi
    promotion_section = soup.find('div', class_='promotion-list')
    if promotion_section:
        promotion_items = promotion_section.find_all('li')
        promotion_text = "; ".join(item.get_text(separator=" ", strip=True) for item in promotion_items)

    return promotion_text

def clean_price(price_str):
    """Chuyển đổi chuỗi giá thành số nguyên."""
    return int(price_str.replace('.', '').replace('đ', '').strip())

def spider(max_pages):
    product_data = []  # List to store all product data
    page = 1
    id_counter = 1

    while page <= max_pages:
        url = 'https://www.xtmobile.vn/dien-thoai/?p=' + str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        for pinfo in soup.find_all('div', class_='pinfo'):
            # Lấy link và tiêu đề sản phẩm
            link_tag = pinfo.find('h3').find('a')
            href = link_tag.get('href')
            title = link_tag.get('title')

            # Lấy giá sản phẩm
            price_tag = pinfo.find('div', class_='price')
            current_price = price_tag.contents[0].strip()
            original_price = current_price

            if price_tag.find('span'):
                original_price = price_tag.find('span').contents[0].strip()

            # Chuyển đổi giá từ chuỗi sang số nguyên
            current_price_value = clean_price(current_price)
            original_price_value = clean_price(original_price)

            # Tính discount price
            discount_price_value = original_price_value - current_price_value

            # Gọi các hàm để lấy metadata, description và promotion
            metadata = get_single_meta_item_data(href)
            description = get_single_desc_item_data(href)
            promotion = get_single_promotion_data(href)

            # Thêm dữ liệu sản phẩm vào danh sách
            product_data.append({
                "id": id_counter,
                "product": title,
                "original_price": original_price_value,
                "discount_price": discount_price_value,
                "metadata": metadata,
                "description": description,
                "promotion": promotion
            })
            id_counter += 1  # Tăng id cho sản phẩm tiếp theo

            # Đợi một chút để tránh bị chặn khi truy xuất dữ liệu nhanh
            time.sleep(1)  # Đợi 1 giây giữa các sản phẩm

        page += 1

    # Chuyển danh sách dữ liệu thành DataFrame
    df = pd.DataFrame(product_data)
    return df

# Gọi hàm và lưu kết quả vào DataFrame
product_df = spider(3)  # Chỉ lấy 3 trang để thử nghiệm
# print(product_df.head(1))


In [19]:
product_df.head(10)

,id,product,original_price,discount_price,metadata,description,promotion
0,1,Samsung Galaxy S24 5G (8GB|256GB) (CTY),22999000,7400000,"Tiện ích Tính năng đặc biệt Samsung DeX, Hỗ tr...",Samsung S24 256GB CTY - Điện thoại dáng mua...,"Thu máy cũ giá cao nhất thị trường, giảm thêm ..."
1,2,Samsung Galaxy S24 Plus 5G (12GB|256GB) (CTY),26990000,9791000,"Tiện ích Tính năng đặc biệt Hỗ trợ 5G, Bảo mật...",Samsung Galaxy S24 Plus 256GB CTY - Chip Exyno...,"Thu máy cũ giá cao nhất thị trường, giảm thêm ..."
2,3,Samsung Galaxy S24 Ultra 5G (12GB|256GB) (CTY),33999000,10400000,"Tiện ích Tính năng đặc biệt Samsung DeX, Hỗ tr...",Samsung Galaxy S24 Ultra 256GB - Mẫu AI phone ...,Hỗ trợ trả góp 0% lãi suất qua thẻ tín dụng; G...
3,4,Samsung Galaxy A55 5G (8GB|256GB) (CTY),9399000,0,Tiện ích Tính năng đặc biệt Chống bụi/nước IP6...,N/A,Giảm 50% khi nâng cấp bảo hành mở rộng 1 đổi 1...
4,5,Samsung Galaxy S23 Ultra (8GB|256GB) (CTY),31990000,11491000,Tiện ích Tính năng đặc biệt Bảo mật Mở khoá vâ...,Samsung S23 Ultra 256GB: Sự lựa chọn mang tính...,Hỗ trợ trả góp 0% lãi suất qua thẻ tín dụng; G...
5,6,Samsung Galaxy A90 5G (6GB|128GB) SM-A908N (Cũ...,5990000,2591000,Tiện ích Tính năng đặc biệt Cảm biến vân tay d...,"Samsung Galaxy A90 5G cũ - Thiết kế cao cấp, m...","Thu cũ lên đời máy mới, giá thu cao nhất..."
6,7,iPhone 16 256GB (VN/A),25999000,1600000,Thông tin màn hình Công nghệ màn hình Super Re...,iPhone 16 256GB (VN/A) - Mẫu iPhone tiêu chuẩn...,Thu cũ đổi mới 90% giá trị máy; Củ sạc Apple ị...
7,8,iPhone 16 Plus 256GB (VN/A),28999000,1200000,Thông tin màn hình Công nghệ màn hình Super Re...,iPhone 16 Plus 256GB - Flagship nhỏ gọn mang đ...,Thu cũ đổi mới 90% giá trị máy; Củ sạc Apple ị...
8,9,iPhone 16 Plus 512GB (Chưa Active),28299000,0,Thông tin màn hình Công nghệ màn hình Super Re...,Đánh giá iPhone 16 Plus 512GB Quốc tế - Chiê...,Thu cũ đổi mới 90% giá trị máy; Củ sạc Apple ị...
9,10,iPhone 16 Pro 256GB (VN/A),31999000,1100000,Thông tin màn hình Công nghệ màn hình 6.3 inch...,iPhone 16 Pro 256GB - Siêu phẩm công nghệ với ...,Thu cũ đổi mới 90% giá trị máy; Củ sạc Apple ị...


In [20]:
from google.colab import files

# Lưu DataFrame vào file CSV
product_df.to_csv('product_data_3_pages.csv', index=False, encoding='utf-8-sig')

# Download tập dữ liệu
files.download('product_data_3_pages.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Convert Data to Json Format:

In [24]:
def convert_raw_data_to_string(row):
    # Chuyển đổi một hàng của DataFrame thành chuỗi, bao gồm metadata
    return (f"Tên sản phẩm: {row['product']}, "
            f"Giá gốc: {row['original_price']}, "
            f"Tiền giảm: {row['discount_price']}, "
            f"Mô tả: {row['description']}, "
            f"Metadata: {row['metadata']}, "
            f"Chương trình giảm giá: {row['promotion']}")

def process_and_save_data_from_df(product_df, output_file):
    schema = schema_english  # Hoặc schema_vietnamese nếu bạn muốn dùng schema tiếng Việt

    processed_data = []

    for index, row in product_df.iterrows():
        # Chuyển đổi hàng DataFrame thành chuỗi
        raw_data = convert_raw_data_to_string(row)

        # Gọi hàm convert_to_json để chuyển đổi
        json_data = convert_to_json(raw_data, schema)

        if json_data:
            processed_data.append(json_data)

    # Lưu dữ liệu đã xử lý vào tệp JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

# Gọi hàm để lưu dữ liệu vào JSON
process_and_save_data_from_df(product_df.head(10), 'product_data_3_pages.json')

In [25]:
import json

# Hàm đọc và in nội dung JSON với định dạng dễ đọc
def pretty_print_json(file_path):
    # Đọc dữ liệu từ file JSON
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # In dữ liệu với định dạng pretty print
    print(json.dumps(data, ensure_ascii=False, indent=4))

# Sau khi đã lưu file product_data_3_pages.json, bạn có thể gọi hàm để in
pretty_print_json('product_data_3_pages.json')


[
    {
        "general information": {
            "name": "Samsung Galaxy S24 5G (8GB|256GB) (CTY)",
            "brand": "Samsung",
            "category": "Smartphone",
            "original_price": 22999000.0,
            "discount_price": 7400000.0,
            "description": "Samsung S24 256GB CTY - Điện thoại đáng mua nhất trong năm 2024. Samsung S24 256GB CTY đã chính thức ra mắt tại sự kiện Galaxy Unpacked đầu tiên trong năm 2024. Chiếc điện thoại ngay lập tức thu hút người dùng nhờ vào viền màn hình siêu mỏng, con chip Exynos 2400 \"cây nhà lá vườn\" cùng loạt tính năng AI thú vị.",
            "design": "Thiết kế thu hút với viền siêu mỏng, có ngoại hình khá giống với thế hệ tiền nhiệm.",
            "promotion": "Giá bán Samsung Galaxy S24 CTY hiện tại từ 23.990.000đ - 27.990.000đ tùy vào phiên bản dung lượng. Đặc biệt, khi tham gia chương trình đặt trước Galaxy S24, bạn sẽ nhận ngay một voucher giảm giá 2.000.000đ cùng gói bảo hành Samsung Care+ trong vòng 2 năm."
      